In [5]:
import zpu

In [62]:
def Ei(i):
    i = int(i)
    ei = [0,0,0]
    ei[i] = str(i)
    ei[(i+1)%3] = str((2+i)%3)
    ei[(i+2)%3] = str((1+i)%3)
    return ei
    

In [76]:
def isEipal(seq, i):
    "kontroluje jestli řetězec je pseudopalindrom"
    ei = Ei(i)
    l = len(seq)
    if l == 1:
        if seq == i:
            return True
        else:
            return False
    for x in range(0, l//2):
        if seq[x] != ei[int(seq[l-1-x])]:
            return False
    return(True)

In [117]:
def makeEipalClosure (seq, i):
    "udělá z řetězce pseudopalindromický uzávěr"
    ei = Ei(i)
    if isEipal(seq, i) == True:
        return(seq)
    j = 1
    while isEipal(seq[j:], i) != True:
        print(seq[j:])
        j = j+1
    #print("    {0} nejdelší pseudopalindromický uzávěr : {1}".format(seq,seq[i:]))
    closure = seq
    pref = seq[j-1::-1]
    for letter in pref:
        closure = closure + ei[int(letter)]
    return(closure)

In [112]:
isEipal("", "2")

True

In [123]:
makeEipalClosure ("2", "0")

'21'

In [115]:
zpu.makeEpalClosure("00")

'0011'

In [116]:
"haha"[4:4]

''